**Feature Engineering**
==========
모델에 직접 적용하기 전에 데이터들을 분석하고 이를 training하기 좋게 변환하는 과정이 중요하다 생각하여 하는 과정임 - Sina의 Kernel 참조하였음

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import re as re
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('C:/Users/sangkyu/git/kaggle/titanic/input/train.csv')
test = pd.read_csv('C:/Users/sangkyu/git/kaggle/titanic/input/test.csv')
full_data = [train,test]
print (train.info())
PassengerId = test['PassengerId']


**Feature Engineering**


1. Pclass : Missing Value가 없고 이미 Numerical Value임. 

In [ ]:
train[['Pclass','Survived']].groupby(['Pclass'],as_index=False).mean()
# as_index=True로 설정하게되면 보기 불편하게 되어있음

2. Sex : 남,녀 남자보다 여자가 생존확률이 높음

In [ ]:
train[['Sex','Survived']].groupby(['Sex'],as_index=False).mean()

3. Age : Missing Value가 많다. 나이를 구간별로 나누어보자

In [ ]:
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
    
train['CategoricalAge'] = pd.cut(train['Age'], 5)

print (train[['CategoricalAge', 'Survived']].groupby(['CategoricalAge'], as_index=False).mean())

4. SibSp, Parch : SibSp는 형제/부부의 수를 나타내고 Parch는 부모/아이의 수를 나타낸다 이걸로 우리는 새로운 feature를 만들 수 있다.

In [ ]:
train[['SibSp','Survived']].groupby(['SibSp'],as_index=False).mean()

In [ ]:
train[['Parch','Survived']].groupby(['Parch'],as_index=False).mean()

In [ ]:
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
train[['FamilySize','Survived']].groupby(['FamilySize'],as_index=False).mean()
# FamilySize라는 Feature가 언뜻보면 생존에 영향을 끼치는 것 같기도 하다.

In [ ]:
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize']==1,'IsAlone']=1
train[['IsAlone','Survived']].groupby(['IsAlone'],as_index=False).mean()
# 혼자 배에 탔는지 안탔는지가 영향을 주는 것 같아 보인다.

5. Embarked : 배를 어디에서 탔는지에 대한 것인데 어떻게 분석하지? 두개의 Missing Value가 있다.

In [ ]:
train.info()

In [ ]:
# 어렵게 생각하지않고 Missing Value를 최빈값으로 채워 넣어야겠다.
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
train[['Embarked','Survived']].groupby(['Embarked'],as_index=False).mean()

In [ ]:
train.info()

6. Fare : 운임으로 결측값은 없다. 4개의 카테고리로 나누어 생존과 관련이 있는지 확인해봄

In [ ]:
train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
# pd.qcut -> 동일한 개수로 나누기
train[['CategoricalFare','Survived']].groupby(['CategoricalFare'],as_index=False).mean()
# 높은 운임을 낸 사람이 조금 더 살아남은것으로 보임

7. Name : 이름에서 Insight를 얻기 위한 과정

In [ ]:
train['Name']
# 보면 이름이 중구난방으로 적혀있어...어떻게할까? 보면 Mr,Miss,Master,Mrs,Rev등으로 나누어짐
# 나는 외국이름을 나누는데 어떻게 나눠야할지 모르겠어..

In [ ]:
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name) or re.search(' ([A-Za-z]+).', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)

print(pd.crosstab(train['Title'], train['Sex']))


# 이름중에서 성만 따로 추출해서 'Title'이라는 새로운 Column을 만듬

In [ ]:
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady','Capt','Col','Countess','Don','Dr','Jonkheer','Major',
                                                'Rev','Sir','Dona'],'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle','Miss')
    dataset['Title'] = dataset['Title'].replace('Mme','Mrs')
    dataset['Title'] = dataset['Title'].replace('Ms','Miss')
    
train[['Title','Survived']].groupby(['Title'],as_index=False).mean()

# 사람이 타이핑 하면서 오타가 난것들을 바꾸어주고 흔하지 않은 성을 Rare로 하나로 묶어 Survived와의 관계를 확인 하였다


Data Cleaning
====
변수들에 대한 전처리를 진행하였고 이제 이 데이터를 깨끗하게 만들자

In [ ]:
for dataset in full_data:
    # 'Sex' : 여자를 1 남자를 0으로 매핑하자
    dataset['Sex'] = dataset['Sex'].replace('female',0)
    dataset['Sex'] = dataset['Sex'].replace('male',1)
    # 'Title' : 5개로 나누어진 'Title'변수를 숫자로 매핑함
    dataset['Title'] = dataset['Title'].replace('Master',1)
    dataset['Title'] = dataset['Title'].replace('Miss',2)
    dataset['Title'] = dataset['Title'].replace('Mr',3)
    dataset['Title'] = dataset['Title'].replace('Mrs',4)
    dataset['Title'] = dataset['Title'].replace('Rare',5)

    # 'Embarked' C,Q,S로 나누어져있음 이걸 숫자로 매핑하자
    dataset['Embarked'] = dataset['Embarked'].replace('C',0)
    dataset['Embarked'] = dataset['Embarked'].replace('Q',1)
    dataset['Embarked'] = dataset['Embarked'].replace('S',2)
    # 'Fare' 구간을 숫자로 매핑하자
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[ (dataset['Fare'] > 7.91) & (dataset['Fare']< 14.454), 'Fare'] = 1
    dataset.loc[ (dataset['Fare'] > 14.454) & (dataset['Fare']< 31.0), 'Fare'] = 2
    dataset.loc[ dataset['Fare'] > 31.0, 'Fare'] = 3
    # 'Age' 구간을 숫자로 매핑하자
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4
# Feature들중에서 필요없는건 지우고 필요한건 선택하여 train set을 완성하자.
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp',
                 'Parch', 'FamilySize']
train = train.drop(drop_elements, axis = 1)
train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)

test  = test.drop(drop_elements, axis = 1)
print(train.shape)
print(test.shape)



    
    

In [ ]:
y=train['Survived']
X=train.drop(columns=['Survived'],axis=1)

In [ ]:
test.fillna(test.mean(),inplace=True)

In [ ]:
test.isnull().sum()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)

In [ ]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(objective='multiclass', random_state=42)

lgbm.fit(X_train, y_train)

y_pred = lgbm.predict(test) 
lgbm.score(X_test,y_test)


In [ ]:
test_2 = pd.read_csv('C:/Users/sangkyu/git/kaggle/titanic/input/test.csv' , header = 0, dtype={'Age': np.float64})

In [ ]:
result = pd.DataFrame(pred,columns=['Survived'])
submission13 = result.join(test_2['PassengerId']).iloc[:,::-1]

In [ ]:
submission13.to_csv('submission13.csv', index=False)